## 载入包

In [1]:
# 添加项目路径到 sys.path
import sys
from pathlib import Path

# 获取 notebook 所在目录,然后向上两级到达项目根目录
# /home/ako/Project/work/EEG/test/train.ipynb -> /home/ako/Project/work/
notebook_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd()
project_root = notebook_dir.parent.parent  # test -> EEG -> work

# 添加项目根目录和 libeer-ako 到 sys.path
libeer_root = project_root / 'lib' / 'libeer-ako'
for path in [str(project_root), str(libeer_root)]:
    if path not in sys.path:
        sys.path.insert(0, path)

# 使用绝对导入
from EEG.utils.Models import Model
from LibEER.config.setting import seed_sub_dependent_front_back_setting, preset_setting, set_setting_by_args
from LibEER.data_utils.load_data import get_data
from LibEER.data_utils.split import merge_to_part, index_to_data, get_split_index
from LibEER.utils.args import get_args_parser
from LibEER.utils.store import make_output_dir
from LibEER.utils.utils import state_log, result_log, setup_seed, sub_result_log
from LibEER.Trainer.training import train
from LibEER.models.DGCNN import NewSparseL2Regularization
import torch
import torch.optim as optim
import torch.nn as nn

print(f"✅ Project root: {project_root}")
print(f"✅ LibEER root: {libeer_root}")
print(f"✅ Successfully imported Model from EEG.utils.Models")

[PGCN_Adapter] Successfully registered PGCN in LibEER.models.Models
✅ Project root: /home/ako/Project/work
✅ LibEER root: /home/ako/Project/work/lib/libeer-ako
✅ Successfully imported Model from EEG.utils.Models


In [12]:
def main(args):
    if args.setting is not None:
        setting = preset_setting[args.setting](args)
    else:
        setting = set_setting_by_args(args)
    setup_seed(args.seed)
    data, label, channels, feature_dim, num_classes = get_data(setting)
    data, label = merge_to_part(data, label, setting)
    device = torch.device(args.device)
    best_metrics = []
    subjects_metrics = [[]for _ in range(len(data))]
    for rridx, (data_i, label_i) in enumerate(zip(data, label), 1):
        tts = get_split_index(data_i, label_i, setting)
        for ridx, (train_indexes, test_indexes, val_indexes) in enumerate(zip(tts['train'], tts['test'], tts['val']), 1):
            setup_seed(args.seed)
            if val_indexes[0] == -1:
                print(f"train indexes:{train_indexes}, test indexes:{test_indexes}")
            else:
                print(f"train indexes:{train_indexes}, val indexes:{val_indexes}, test indexes:{test_indexes}")

            # split train and test data by specified experiment mode
            train_data, train_label, val_data, val_label, test_data, test_label = \
                index_to_data(data_i, label_i, train_indexes, test_indexes, val_indexes, args.keep_dim)
            # model to train
            if len(val_data) == 0:
                val_data = test_data
                val_label = test_label

            model = Model['DGCNN'](channels, feature_dim, num_classes)
            # Train one round using the train one round function defined in the model
            dataset_train = torch.utils.data.TensorDataset(torch.Tensor(train_data), torch.Tensor(train_label))
            dataset_val = torch.utils.data.TensorDataset(torch.Tensor(val_data), torch.Tensor(val_label))
            dataset_test = torch.utils.data.TensorDataset(torch.Tensor(test_data), torch.Tensor(test_label))
            optimizer = optim.AdamW(model.parameters(), lr=args.lr, weight_decay=1e-4, eps=1e-4)
            criterion = nn.CrossEntropyLoss()
            loss_func = NewSparseL2Regularization(0.01).to(device)
            output_dir = make_output_dir(args, "DGCNN")
            round_metric = train(model=model, dataset_train=dataset_train, dataset_val=dataset_val, dataset_test=dataset_test, device=device,
                                 output_dir=output_dir, metrics=args.metrics, metric_choose=args.metric_choose, optimizer=optimizer,
                                 batch_size=args.batch_size, epochs=args.epochs, criterion=criterion, loss_func=loss_func, loss_param=model)
            best_metrics.append(round_metric)
            if setting.experiment_mode == "subject-dependent":
                subjects_metrics[rridx-1].append(round_metric)
    # best metrics: every round metrics dict
    # subjects metrics: (subject, sub_round_metric)
    if setting.experiment_mode == "subject-dependent":
        sub_result_log(args, subjects_metrics)
    else:
        result_log(args, best_metrics)


In [13]:
if __name__ == '__main__':
    args = get_args_parser()
    args = args.parse_args()
    # log out train state
    state_log(args)
    main(args)


usage: EEG Lib for emotion recognition based on EEG [-batch_size BATCH_SIZE]
                                                    [-epochs EPOCHS]
                                                    [-device {cuda,cpu}]
                                                    [-eval] [-seed SEED]
                                                    [-num_workers NUM_WORKERS]
                                                    [-loss_func LOSS_FUNC]
                                                    [-metrics METRICS [METRICS ...]]
                                                    [-metric_choose METRIC_CHOOSE]
                                                    [-lr LR]
                                                    [-data_dir DATA_DIR]
                                                    [-resume]
                                                    [-resume_epoch RESUME_EPOCH]
                                                    [-checkpoint CHECKPOINT]
                                

SystemExit: 2

## DGCNN Layer [64] 实验

测试 DGCNN 在 SEED 数据集上使用 `layers=[64]` 配置的效果

In [ ]:
# 创建参数配置
class Args:
    def __init__(self):
        # 数据集配置
        self.dataset = 'seed_de_lds'
        self.dataset_path = '../../datasets/SEED'
        self.setting = 'seed_sub_dependent_train_val_test_setting'
        self.feature_type = 'de_lds'  # ✅ 添加
        self.time_window = 1           # ✅ 添加
        
        # 训练配置
        self.model = 'DGCNN'
        self.batch_size = 32
        self.epochs = 80
        self.lr = 0.0015
        
        # 评估指标
        self.metrics = ['acc', 'macro-f1']
        self.metric_choose = 'macro-f1'
        
        # 其他配置
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.seed = 2024
        self.sessions = [1, 2, 3]  # 使用全部 3 个 sessions
        self.keep_dim = False
        self.onehot = True
        
        # 标签配置
        self.bounds = None             # ✅ 添加
        self.label_used = None         # ✅ 添加
        
        # 输出目录
        self.log_dir = './log/'
        self.output_dir = './result/'

args = Args()
print(f"✅ 配置完成:")
print(f"  模型: {args.model}")
print(f"  数据集: {args.dataset}")
print(f"  特征类型: {args.feature_type}")
print(f"  Sessions: {args.sessions}")
print(f"  Batch size: {args.batch_size}")
print(f"  Learning rate: {args.lr}")
print(f"  Epochs: {args.epochs}")
print(f"  Device: {args.device}")

✅ 配置完成:
  模型: DGCNN
  数据集: seed_de_lds
  Sessions: [1, 2, 3]
  Batch size: 32
  Learning rate: 0.0015
  Epochs: 80
  Device: cuda


### 查看 DGCNN 网络结构

分析 DGCNN 的每一层配置:输入通道、输出通道、参数量

In [2]:
# 创建一个 DGCNN 模型实例来查看结构
# SEED 数据集参数
num_electrodes = 62  # 电极数
in_channels = 5      # 特征维度 (DE_LDS: delta, theta, alpha, beta, gamma)
num_classes = 3      # 类别数 (positive, neutral, negative)

# 创建模型（禁用 yaml 配置输出）
import io
import sys
old_stdout = sys.stdout
sys.stdout = io.StringIO()  # 临时重定向输出
dgcnn_model = Model['DGCNN'](num_electrodes, in_channels, num_classes)
sys.stdout = old_stdout  # 恢复输出

# 核心配置
print("=" * 70)
print("           DGCNN 网络结构")
print("=" * 70)

print("\n⭐ 图卷积层配置:")
print(f"   层数:      {len(dgcnn_model.layers)} 层")
print(f"   通道数:    {dgcnn_model.layers}")
print(f"   Chebyshev: {dgcnn_model.k} 阶多项式")
print(f"   Dropout:   {dgcnn_model.dropout_rate}")

print("\n📊 输入输出:")
print(f"   输入: {num_electrodes} 电极 × {in_channels} 特征")
print(f"   输出: {num_classes} 类别")

# 数据流
print("\n🔄 数据流:")
print(f"   输入:  (batch, {num_electrodes}, {in_channels})")

for i, layer_ch in enumerate(dgcnn_model.layers):
    conv = dgcnn_model.graphConvs[i]
    params = conv.weight.numel()
    print(f"    ↓ GraphConv{i+1}: {conv.in_channels}→{conv.out_channels} 通道 ({params} 参数)")

last_out = num_electrodes * dgcnn_model.layers[-1]
print(f"   (batch, {num_electrodes}, {dgcnn_model.layers[-1]})")
print(f"    ↓ Flatten")
print(f"   (batch, {last_out})")

fc1_params = dgcnn_model.fc.weight.numel() + dgcnn_model.fc.bias.numel()
print(f"    ↓ FC1: {last_out}→256 ({fc1_params:,} 参数)")
print(f"   (batch, 256)")

fc2_params = dgcnn_model.fc2.weight.numel() + dgcnn_model.fc2.bias.numel()
print(f"    ↓ FC2: 256→3 ({fc2_params} 参数)")
print(f"   (batch, 3)")

# 参数统计
print("\n📊 参数统计:")
conv_params = sum(c.weight.numel() for c in dgcnn_model.graphConvs)
brelu_params = sum(b.bias.numel() for b in dgcnn_model.b_relus)
fc_params = fc1_params + fc2_params
adj_params = dgcnn_model.adj.numel() + dgcnn_model.adj_bias.numel()
total = sum(p.numel() for p in dgcnn_model.parameters())

print(f"   GraphConv:    {conv_params:>8,}")
print(f"   B-ReLU:       {brelu_params:>8,}")
print(f"   全连接:       {fc_params:>8,}")
print(f"   邻接矩阵:     {adj_params:>8,} (62×62)")
print(f"   " + "-" * 22)
print(f"   总计:         {total:>8,} ({total*4/1024/1024:.2f} MB)")

print("=" * 70)

           DGCNN 网络结构

⭐ 图卷积层配置:
   层数:      1 层
   通道数:    [64]
   Chebyshev: 2 阶多项式
   Dropout:   0.5

📊 输入输出:
   输入: 62 电极 × 5 特征
   输出: 3 类别

🔄 数据流:
   输入:  (batch, 62, 5)
    ↓ GraphConv1: 5→64 通道 (640 参数)
   (batch, 62, 64)
    ↓ Flatten
   (batch, 3968)
    ↓ FC1: 3968→256 (1,016,064 参数)
   (batch, 256)
    ↓ FC2: 256→3 (771 参数)
   (batch, 3)

📊 参数统计:
   GraphConv:         640
   B-ReLU:             64
   全连接:       1,016,835
   邻接矩阵:        3,845 (62×62)
   ----------------------
   总计:         1,021,384 (3.90 MB)


In [15]:
# 训练 DGCNN 模型 (layers=[64])
# 这将自动使用 DGCNN 的默认配置: 62 电极 -> layers=[64]

print("="*60)
print("开始训练 DGCNN (layers=[64])")
print("="*60)

# 运行训练
state_log(args)
main(args)

print("\n" + "="*60)
print("训练完成!")
print("="*60)

开始训练 DGCNN (layers=[64])


AttributeError: 'Args' object has no attribute 'feature_type'

### 预期结果

根据 DGCNN_train.py 中的注释:
```
python DGCNN_train.py -metrics 'acc' 'macro-f1' -metric_choose 'macro-f1' 
  -setting seed_sub_dependent_train_val_test_setting 
  -dataset seed_de_lds -batch_size 32 -seed 2024 -epochs 80 -lr 0.0015 -onehot
```

**预期准确率**: 0.8255 ± 0.1561 (acc) / 0.7989 ± 0.1893 (macro-f1)

这个配置在 SEED 数据集上:
- 使用 62 个电极 → `layers=[64]` (自动设置)
- Subject-dependent 模式
- Train/Val/Test 三分划分
- 3 个 sessions 的数据